In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

In [2]:
TRAIN_DIR = './train'
TEST_DIR = './test'
# 50x50px image
IMG_SIZE = 50
# learning rate = 0.001
LR = 1e-3

In [3]:
def label_to_one_hot(label):
    # image labels are in format 'dog.123123.jpg'
    word_label = label.split('.')[0]
    if word_label == 'cat': return [1, 0]
    elif word_label == 'dog': return [0, 1]

In [4]:
def create_train_data(grayscale=False):
    train_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_to_one_hot(img)
        path = os.path.join(TRAIN_DIR, img)
        mode = 0 if grayscale else 1
        img = cv2.resize(cv2.imread(path, mode), (IMG_SIZE, IMG_SIZE))
        train_data.append([np.array(img), np.array(label)])
    shuffle(train_data)
    np.save('train_data.npy', train_data)
    return train_data

In [5]:
if (os.path.isfile('train_data.npy')):
    train_data = np.load('train_data.npy')
else:
    train_data = create_train_data()
# X - images
X = np.array([i[0] for i in train_data])
# Y - 'one_hot' labels
Y = np.array([i[1] for i in train_data])
# reshape to: [num_images, IMG_SIZE, IMG_SIZE, num_channels(3 for color, 1 for grayscale)]
X = X.reshape(X.shape[0], IMG_SIZE, IMG_SIZE, 3)
print('RGB values of image 0, row 0, column 0: ', X[0][0][0])
# since the values are integers, let's convert them to float
X = X.astype('float32')
# let's normalize the values, initially they are in range (0,255)
# we convert them to range (-0.5, 0.5), so their mean will be 0
X = (X - 127) / 255
print('(num_images, IMG_SIZE, IMG_SIZE, num_channels)')
print(X.shape)

RGB values of image 0, row 0, column 0:  [ 88 100 104]
(num_images, IMG_SIZE, IMG_SIZE, num_channels)
(25000, 50, 50, 3)


In [6]:
# let's use 10% of our data for testing
# take last 2500 elements
x_test = X[-2500:]
y_test = Y[-2500:]
x_train = X[:-2500]
y_train = Y[:-2500]
print('train size:', x_train.shape[0])
print('test size:', x_test.shape[0])

train size: 22500
test size: 2500


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# output size = num of classes
model.add(Dense(2, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="adam", metrics=['accuracy'])
tbCallBack = TensorBoard(log_dir='./log/model-adam-c5128', histogram_freq=0,  
          write_graph=True, write_images=True)
model.fit(x_train, y_train,
          batch_size=100,
          epochs=12,
          verbose=2,
          validation_split=0.2,
          callbacks=[tbCallBack])
# let's evaluate model on test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 18000 samples, validate on 4500 samples
Epoch 1/12
 - 26s - loss: 0.6336 - acc: 0.6319 - val_loss: 0.5696 - val_acc: 0.7082
Epoch 2/12
 - 25s - loss: 0.5500 - acc: 0.7197 - val_loss: 0.5468 - val_acc: 0.7262
Epoch 3/12
 - 25s - loss: 0.4904 - acc: 0.7639 - val_loss: 0.5006 - val_acc: 0.7567
Epoch 4/12
 - 25s - loss: 0.4480 - acc: 0.7880 - val_loss: 0.4223 - val_acc: 0.8029
Epoch 5/12
 - 25s - loss: 0.4039 - acc: 0.8168 - val_loss: 0.4108 - val_acc: 0.8160
Epoch 6/12
 - 25s - loss: 0.3643 - acc: 0.8374 - val_loss: 0.3759 - val_acc: 0.8331
Epoch 7/12
 - 25s - loss: 0.3405 - acc: 0.8482 - val_loss: 0.3694 - val_acc: 0.8353
Epoch 8/12
 - 25s - loss: 0.3102 - acc: 0.8649 - val_loss: 0.3646 - val_acc: 0.8444
Epoch 9/12
 - 25s - loss: 0.2857 - acc: 0.8744 - val_loss: 0.3724 - val_acc: 0.8387
Epoch 10/12
 - 25s - loss: 0.2605 - acc: 0.8869 - val_loss: 0.3587 - val_acc: 0.8469
Epoch 11/12
 - 25s - loss: 0.2244 - acc: 0.9043 - val_loss: 0.3662 - val_acc: 0.8436
Epoch 12/12
 - 25s - loss